In [15]:
import os
import numpy as np 
import pandas as pd 
from mne_connectivity import spectral_connectivity_time

In [16]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Connectivity')
#%run connectivity.py

os.chdir('/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Preprocessing/')
%run constants.py
%run filter.py
%run load_files.py

directory_path = '/home/melissa/PREPROCESSING/SYNGAP1/numpyformat_baseline/'
results_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Connectivity/'
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
channel_labels = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15]

In [19]:
class ConnectivityClass:
    def __init__(self, filtered_data, channels, animal_id):
        self.filtered_data = filtered_data
        self.channels = channels
        self.animal_id = animal_id
        
    def channel_pairs(self):
        pairs = []
        for i in range(len(self.channels)):
            for j in range(i + 1, len(self.channels)):
                pairs.append([self.channels[i], self.channels[j]])
        return pairs

    def calculate_max_cross_corr(self, num_epochs, freq_band):
        cross_corr_ls = []
        error_ls = []
        for i in range(num_epochs):
            try:
                one_epoch = compute_max_cross_corr(sfreq=250.4, data=self.filtered_data[:, i])
                df = pd.DataFrame([one_epoch], columns=[f'{ch[0]}_{ch[1]}_{freq_band}' for ch in self.channel_pairs()])
                df_other = pd.DataFrame(data = {'Epoch': [i], 'Animal_ID': [self.animal_id]})
                df_concat = pd.concat([df_other, df], axis = 1)
                cross_corr_ls.append(df_concat)
            except Exception as e:
                print(f'Error for index {i}: {e}')
                error_ls.append(i)

        cross_corr_concat = pd.concat(cross_corr_ls)
        error_array = np.array(error_ls)
        return cross_corr_concat, error_array
    
    def calculate_plv_mne(self, filtered_data, freq_band, freq_name):
        tr_filter = filtered_data.transpose(1, 0, 2)
        connectivity_array = spectral_connectivity_time(tr_filter, freqs=freq_band, n_cycles= 3,
                                                        method= 'plv', average=False, sfreq=250.4, fmin=freq_band[0],
                                                        fmax=freq_band[1], faverage=True).get_data()
        all_epochs = []
        for epoch in connectivity_array:
            df = pd.DataFrame([epoch], columns=[f'{ch[0]}_{ch[1]}_{freq_name}' for ch in self.channel_pairs()])
            all_epochs.append(df)
        
        concat_epochs = pd.concat(all_epochs)
        id_df = pd.DataFrame(data = {'Animal_ID': [self.animal_id]*len(filtered_data[0]), 'Epoch': np.arange(0, len(filtered_data[0]), 1) })
        anim_df = pd.concat([id_df, concat_epochs], axis = 1)
        return anim_df

In [ ]:
for animal in analysis_ls:
    print(f'loading {animal}')
    animal = str(animal)
    load_files = LoadFiles(directory_path, animal)
    if animal in SYNGAP_2_ls:
        data_1, data_2, brain_state_1, brain_state_2 = load_files.load_two_analysis_files(start_times_dict=SYNGAP_baseline_start, end_times_dict=SYNGAP_baseline_end)
        data = np.concatenate([data_1, data_2], axis = 1)
        frequency_bands = [(1, 5), (5, 11), (11, 16), (16, 30), (30, 48)]
        frequency_names = ['delta', 'theta', 'sigma', 'beta', 'gamma']
        connectivity_cal = 'phase_lock'
        freq_results = []
        for (low, high), label in zip(frequency_bands, frequency_names):
            print(f'Processing {label} band: {low}-{high} Hz')
            noise_filter = NoiseFilter(data, brain_state_file=brain_state_1, channelvariables=channel_variables, ch_type='eeg')
            bandpass_filtered_data = noise_filter.filter_data_type()
            filtered_data = np.moveaxis(np.array(np.split(bandpass_filtered_data, 34560, axis=1)), 1, 0)
            complexity_calculations = ConnectivityClass(filtered_data, channels = channel_labels, animal_id = animal)
            if connectivity_cal == 'phase_lock':
                df_result = complexity_calculations.calculate_plv_mne(filtered_data = filtered_data, 
                                                                                 freq_band = (low, high), 
                                                                                freq_name = label)
                    
                freq_results.append(df_result)
                break
            break
        break

loading S7088
loading S7092
loading S7086
Processing delta band: 1-5 Hz
only using indices for lower-triangular matrix
Connectivity computation...
   Processing epoch 1 / 34560 ...
   Processing epoch 2 / 34560 ...
   Processing epoch 3 / 34560 ...
   Processing epoch 4 / 34560 ...
   Processing epoch 5 / 34560 ...
   Processing epoch 6 / 34560 ...
   Processing epoch 7 / 34560 ...
   Processing epoch 8 / 34560 ...
   Processing epoch 9 / 34560 ...
   Processing epoch 10 / 34560 ...
   Processing epoch 11 / 34560 ...
   Processing epoch 12 / 34560 ...
   Processing epoch 13 / 34560 ...
   Processing epoch 14 / 34560 ...
   Processing epoch 15 / 34560 ...
   Processing epoch 16 / 34560 ...
   Processing epoch 17 / 34560 ...
   Processing epoch 18 / 34560 ...
   Processing epoch 19 / 34560 ...
   Processing epoch 20 / 34560 ...
   Processing epoch 21 / 34560 ...
   Processing epoch 22 / 34560 ...
   Processing epoch 23 / 34560 ...
   Processing epoch 24 / 34560 ...
   Processing epoch 25

   Processing epoch 228 / 34560 ...
   Processing epoch 229 / 34560 ...
   Processing epoch 230 / 34560 ...
   Processing epoch 231 / 34560 ...
   Processing epoch 232 / 34560 ...
   Processing epoch 233 / 34560 ...
   Processing epoch 234 / 34560 ...
   Processing epoch 235 / 34560 ...
   Processing epoch 236 / 34560 ...
   Processing epoch 237 / 34560 ...
   Processing epoch 238 / 34560 ...
   Processing epoch 239 / 34560 ...
   Processing epoch 240 / 34560 ...
   Processing epoch 241 / 34560 ...
   Processing epoch 242 / 34560 ...
   Processing epoch 243 / 34560 ...
   Processing epoch 244 / 34560 ...
   Processing epoch 245 / 34560 ...
   Processing epoch 246 / 34560 ...
   Processing epoch 247 / 34560 ...
   Processing epoch 248 / 34560 ...
   Processing epoch 249 / 34560 ...
   Processing epoch 250 / 34560 ...
   Processing epoch 251 / 34560 ...
   Processing epoch 252 / 34560 ...
   Processing epoch 253 / 34560 ...
   Processing epoch 254 / 34560 ...
   Processing epoch 255 / 34

   Processing epoch 456 / 34560 ...
   Processing epoch 457 / 34560 ...
   Processing epoch 458 / 34560 ...
   Processing epoch 459 / 34560 ...
   Processing epoch 460 / 34560 ...
   Processing epoch 461 / 34560 ...
   Processing epoch 462 / 34560 ...
   Processing epoch 463 / 34560 ...
   Processing epoch 464 / 34560 ...
   Processing epoch 465 / 34560 ...
   Processing epoch 466 / 34560 ...
   Processing epoch 467 / 34560 ...
   Processing epoch 468 / 34560 ...
   Processing epoch 469 / 34560 ...
   Processing epoch 470 / 34560 ...
   Processing epoch 471 / 34560 ...
   Processing epoch 472 / 34560 ...
   Processing epoch 473 / 34560 ...
   Processing epoch 474 / 34560 ...
   Processing epoch 475 / 34560 ...
   Processing epoch 476 / 34560 ...
   Processing epoch 477 / 34560 ...
   Processing epoch 478 / 34560 ...
   Processing epoch 479 / 34560 ...
   Processing epoch 480 / 34560 ...
   Processing epoch 481 / 34560 ...
   Processing epoch 482 / 34560 ...
   Processing epoch 483 / 34

   Processing epoch 684 / 34560 ...
   Processing epoch 685 / 34560 ...
   Processing epoch 686 / 34560 ...
   Processing epoch 687 / 34560 ...
   Processing epoch 688 / 34560 ...
   Processing epoch 689 / 34560 ...
   Processing epoch 690 / 34560 ...
   Processing epoch 691 / 34560 ...
   Processing epoch 692 / 34560 ...
   Processing epoch 693 / 34560 ...
   Processing epoch 694 / 34560 ...
   Processing epoch 695 / 34560 ...
   Processing epoch 696 / 34560 ...
   Processing epoch 697 / 34560 ...
   Processing epoch 698 / 34560 ...
   Processing epoch 699 / 34560 ...
   Processing epoch 700 / 34560 ...
   Processing epoch 701 / 34560 ...
   Processing epoch 702 / 34560 ...
   Processing epoch 703 / 34560 ...
   Processing epoch 704 / 34560 ...
   Processing epoch 705 / 34560 ...
   Processing epoch 706 / 34560 ...
   Processing epoch 707 / 34560 ...
   Processing epoch 708 / 34560 ...
   Processing epoch 709 / 34560 ...
   Processing epoch 710 / 34560 ...
   Processing epoch 711 / 34

   Processing epoch 912 / 34560 ...
   Processing epoch 913 / 34560 ...
   Processing epoch 914 / 34560 ...
   Processing epoch 915 / 34560 ...
   Processing epoch 916 / 34560 ...
   Processing epoch 917 / 34560 ...
   Processing epoch 918 / 34560 ...
   Processing epoch 919 / 34560 ...
   Processing epoch 920 / 34560 ...
   Processing epoch 921 / 34560 ...
   Processing epoch 922 / 34560 ...
   Processing epoch 923 / 34560 ...
   Processing epoch 924 / 34560 ...
   Processing epoch 925 / 34560 ...
   Processing epoch 926 / 34560 ...
   Processing epoch 927 / 34560 ...
   Processing epoch 928 / 34560 ...
   Processing epoch 929 / 34560 ...
   Processing epoch 930 / 34560 ...
   Processing epoch 931 / 34560 ...
   Processing epoch 932 / 34560 ...
   Processing epoch 933 / 34560 ...
   Processing epoch 934 / 34560 ...
   Processing epoch 935 / 34560 ...
   Processing epoch 936 / 34560 ...
   Processing epoch 937 / 34560 ...
   Processing epoch 938 / 34560 ...
   Processing epoch 939 / 34

...

   Processing epoch 29024 / 34560 ...
   Processing epoch 29025 / 34560 ...
   Processing epoch 29026 / 34560 ...
   Processing epoch 29027 / 34560 ...
   Processing epoch 29028 / 34560 ...
   Processing epoch 29029 / 34560 ...
   Processing epoch 29030 / 34560 ...
   Processing epoch 29031 / 34560 ...
   Processing epoch 29032 / 34560 ...
   Processing epoch 29033 / 34560 ...
   Processing epoch 29034 / 34560 ...
   Processing epoch 29035 / 34560 ...
   Processing epoch 29036 / 34560 ...
   Processing epoch 29037 / 34560 ...
   Processing epoch 29038 / 34560 ...
   Processing epoch 29039 / 34560 ...
   Processing epoch 29040 / 34560 ...
   Processing epoch 29041 / 34560 ...
   Processing epoch 29042 / 34560 ...
   Processing epoch 29043 / 34560 ...
   Processing epoch 29044 / 34560 ...
   Processing epoch 29045 / 34560 ...
   Processing epoch 29046 / 34560 ...
   Processing epoch 29047 / 34560 ...
   Processing epoch 29048 / 34560 ...
   Processing epoch 29049 / 34560 ...
   Processin

   Processing epoch 29240 / 34560 ...
   Processing epoch 29241 / 34560 ...
   Processing epoch 29242 / 34560 ...
   Processing epoch 29243 / 34560 ...
   Processing epoch 29244 / 34560 ...
   Processing epoch 29245 / 34560 ...
   Processing epoch 29246 / 34560 ...
   Processing epoch 29247 / 34560 ...
   Processing epoch 29248 / 34560 ...
   Processing epoch 29249 / 34560 ...
   Processing epoch 29250 / 34560 ...
   Processing epoch 29251 / 34560 ...
   Processing epoch 29252 / 34560 ...
   Processing epoch 29253 / 34560 ...
   Processing epoch 29254 / 34560 ...
   Processing epoch 29255 / 34560 ...
   Processing epoch 29256 / 34560 ...
   Processing epoch 29257 / 34560 ...
   Processing epoch 29258 / 34560 ...
   Processing epoch 29259 / 34560 ...
   Processing epoch 29260 / 34560 ...
   Processing epoch 29261 / 34560 ...
   Processing epoch 29262 / 34560 ...
   Processing epoch 29263 / 34560 ...
   Processing epoch 29264 / 34560 ...
   Processing epoch 29265 / 34560 ...
   Processin

   Processing epoch 29456 / 34560 ...
   Processing epoch 29457 / 34560 ...
   Processing epoch 29458 / 34560 ...
   Processing epoch 29459 / 34560 ...
   Processing epoch 29460 / 34560 ...
   Processing epoch 29461 / 34560 ...
   Processing epoch 29462 / 34560 ...
   Processing epoch 29463 / 34560 ...
   Processing epoch 29464 / 34560 ...
   Processing epoch 29465 / 34560 ...
   Processing epoch 29466 / 34560 ...
   Processing epoch 29467 / 34560 ...
   Processing epoch 29468 / 34560 ...
   Processing epoch 29469 / 34560 ...
   Processing epoch 29470 / 34560 ...
   Processing epoch 29471 / 34560 ...
   Processing epoch 29472 / 34560 ...
   Processing epoch 29473 / 34560 ...
   Processing epoch 29474 / 34560 ...
   Processing epoch 29475 / 34560 ...
   Processing epoch 29476 / 34560 ...
   Processing epoch 29477 / 34560 ...
   Processing epoch 29478 / 34560 ...
   Processing epoch 29479 / 34560 ...
   Processing epoch 29480 / 34560 ...
   Processing epoch 29481 / 34560 ...
   Processin

   Processing epoch 29672 / 34560 ...
   Processing epoch 29673 / 34560 ...
   Processing epoch 29674 / 34560 ...
   Processing epoch 29675 / 34560 ...
   Processing epoch 29676 / 34560 ...
   Processing epoch 29677 / 34560 ...
   Processing epoch 29678 / 34560 ...
   Processing epoch 29679 / 34560 ...
   Processing epoch 29680 / 34560 ...
   Processing epoch 29681 / 34560 ...
   Processing epoch 29682 / 34560 ...
   Processing epoch 29683 / 34560 ...
   Processing epoch 29684 / 34560 ...
   Processing epoch 29685 / 34560 ...
   Processing epoch 29686 / 34560 ...
   Processing epoch 29687 / 34560 ...
   Processing epoch 29688 / 34560 ...
   Processing epoch 29689 / 34560 ...
   Processing epoch 29690 / 34560 ...
   Processing epoch 29691 / 34560 ...
   Processing epoch 29692 / 34560 ...
   Processing epoch 29693 / 34560 ...
   Processing epoch 29694 / 34560 ...
   Processing epoch 29695 / 34560 ...
   Processing epoch 29696 / 34560 ...
   Processing epoch 29697 / 34560 ...
   Processin

   Processing epoch 29888 / 34560 ...
   Processing epoch 29889 / 34560 ...
   Processing epoch 29890 / 34560 ...
   Processing epoch 29891 / 34560 ...
   Processing epoch 29892 / 34560 ...
   Processing epoch 29893 / 34560 ...
   Processing epoch 29894 / 34560 ...
   Processing epoch 29895 / 34560 ...
   Processing epoch 29896 / 34560 ...
   Processing epoch 29897 / 34560 ...
   Processing epoch 29898 / 34560 ...
   Processing epoch 29899 / 34560 ...
   Processing epoch 29900 / 34560 ...
   Processing epoch 29901 / 34560 ...
   Processing epoch 29902 / 34560 ...
   Processing epoch 29903 / 34560 ...
   Processing epoch 29904 / 34560 ...
   Processing epoch 29905 / 34560 ...
   Processing epoch 29906 / 34560 ...
   Processing epoch 29907 / 34560 ...
   Processing epoch 29908 / 34560 ...
   Processing epoch 29909 / 34560 ...
   Processing epoch 29910 / 34560 ...
   Processing epoch 29911 / 34560 ...
   Processing epoch 29912 / 34560 ...
   Processing epoch 29913 / 34560 ...
   Processin

In [11]:
test = np.arange(0, len(filtered_data[0]), 1)

In [12]:
len(test)

34560

In [ ]:
for animal in analysis_ls:
    print(f'loading {animal}')
    animal = str(animal)
    load_files = LoadFiles(directory_path, animal)
    if animal in SYNGAP_2_ls:
        data_1, data_2, brain_state_1, brain_state_2 = load_files.load_two_analysis_files(start_times_dict=SYNGAP_baseline_start, end_times_dict=SYNGAP_baseline_end)
        data_list = [(data_1, brain_state_1), (data_2, brain_state_2)]
        frequency_bands = [(1, 5), (5, 11), (11, 16), (16, 30), (30, 48)]
        frequency_names = ['delta', 'theta', 'sigma', 'beta', 'gamma']
        connectivity_cal = 'cross_corr'
        results = []
        for data, brain_state in data_list:
            freq_results = []
            for (low, high), label in zip(frequency_bands, frequency_names):
                print(f'Processing {label} band: {low}-{high} Hz')
                noise_filter = NoiseFilter(data, brain_state_file=brain_state, channelvariables=channel_variables, ch_type='eeg')
                bandpass_filtered_data = noise_filter.specify_filter(low=low, high=high)
                filtered_data = np.moveaxis(np.array(np.split(bandpass_filtered_data, 17280, axis=1)), 1, 0)
                complexity_calculations = ConnectivityClass(filtered_data, channels = channel_labels, animal_id = animal)
                if connectivity_cal == 'cross_corr':
                    df_result, error = complexity_calculations.calculate_max_cross_corr(num_epochs=17280, freq_band = label)
                    freq_results.append(df_result)
                if connectivity_cal == 'phase_lock':
                    
            freq_concat = pd.concat(freq_results)
            results.append(freq_concat) 
        all_frequencies_concat = pd.concat(results, axis = 1)
        all_frequencies_concat.to_csv(os.path.join(results_path, f'{animal}_{connectivity_cal}.csv'))
    else:
        pass

In [ ]:
elif animal in SYNGAP_1_ls:
        data, brain_state = load_files.load_one_analysis_file(start_times_dict=SYNGAP_baseline_start, end_times_dict=SYNGAP_baseline_end)
        data_list = [(data, brain_state)]
        frequency_bands = [(1, 5), (5, 11), (11, 16), (16, 30), (30, 48)]
        frequency_names = ['delta', 'theta', 'sigma', 'beta', 'gamma']
        connectivity_cal = 'cross_corr'
        results = []
        for data, brain_state in data_list:
            freq_results = []
            for (low, high), label in zip(frequency_bands, frequency_names):
                print(f'Processing {label} band: {low}-{high} Hz')
                noise_filter = NoiseFilter(data, brain_state_file=brain_state, channelvariables=channel_variables, ch_type='eeg')
                bandpass_filtered_data = noise_filter.specify_filter(low=low, high=high)
                filtered_data = np.moveaxis(np.array(np.split(bandpass_filtered_data, 17280, axis=1)), 1, 0)
                complexity_calculations = ConnectivityClass(filtered_data, channels = channel_labels, animal_id = animal)
                if connectivity_cal == 'cross_corr':
                    df_result, error = complexity_calculations.calculate_max_cross_corr(num_epochs=17280, freq_band = label)
                    freq_results.append(df_result)
                
            freq_concat = pd.concat(freq_results)
            results.append(freq_concat) 
        all_frequencies_concat = pd.concat(results, axis = 1)
        all_frequencies_concat.to_csv(os.path.join(results_path, f'{animal}_{connectivity_cal}.csv'))

In [ ]:
def process_data_mcc_plv(animal, connectivity_cal, results_path, is_single_file=False):
    print(f'Loading data for {animal}')
    
    frequency_bands = [(1, 5), (5, 11), (11, 16), (16, 30), (30, 48)]
    frequency_names = ['delta', 'theta', 'sigma', 'beta', 'gamma']
    
    results = []
    for data, brain_state in data_list:
        for (low, high), label in zip(frequency_bands, frequency_names):
            print(f'Processing {label} band: {low}-{high} Hz')
            noise_filter = NoiseFilter(data, brain_state_file=brain_state, channelvariables=channel_variables, ch_type='eeg')
            bandpass_filtered_data = noise_filter.specify_filter(low=low, high=high)
            filtered_data = np.moveaxis(np.array(np.split(bandpass_filtered_data, 17280, axis=1)), 1, 0)
            complexity_calculations = ConnectivityClass(filtered_data)

            if connectivity_cal == 'cross_corr':
                result, error = complexity_calculations.calculate_max_cross_corr(num_epochs=17280)
            elif connectivity_cal == 'phase_lock_val':
                result, error = complexity_calculations.calculate_phase_lock_value(num_epochs=17280)

            results.append((result, error))

        for idx, (result, error) in enumerate(results):
            label = frequency_names[idx % len(frequency_names)]
            if is_single_file or idx < len(results) / 2:
                save_path = f'{results_path}{animal}_{connectivity_cal}_{label}.npy'
                np.save(save_path, result)
                print(f'Saved {save_path}')
            else:
                concatenated_result = np.concatenate([results[i][0] for i in range(idx - len(frequency_names), idx + 1, len(frequency_names))], axis=0)
                save_path = f'{results_path}{animal}_{connectivity_cal}_{label}_concatenated.npy'
                np.save(save_path, concatenated_result)
                print(f'Saved {save_path}')

    print('Processing complete')

def process_data_mne_connect(animal, conn_mes, data, brain_state, frequency_bands, frequency_names, results_path, sfreq=250.4, n_cycles=3):
    noise_filter = NoiseFilter(data, brain_state_file=brain_state, channelvariables=channel_variables, ch_type='eeg')
    bandpass_filtered_data = noise_filter.filter_data_type()
    filter_data = np.moveaxis(np.array(np.split(bandpass_filtered_data, 17280, axis=1)), 1, 0)
    tr_filter = filter_data.transpose(1, 0, 2)
    
    for freq_band, freq_name in zip(frequency_bands, frequency_names):
        connectivity_array = spectral_connectivity_time(tr_filter, freqs=freq_band, n_cycles=n_cycles,
                                                        method=conn_mes, average=False, sfreq=sfreq,
                                                        faverage=True).get_data()
        save_path = results_path / f'{animal}_{conn_mes}_{freq_name}.npy'
        np.save(save_path, connectivity_array)
